In [4]:
from USTs import USTs
import datetime
from datetime import timedelta
from typing import List
from dateutil.relativedelta import relativedelta

In [5]:
# Bill pricing
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [23]:
issue_date = datetime.date(2011, 11, 15)
maturity_date = datetime.date(2041, 11, 15)
as_of_date = datetime.date(2024, 9, 22)

def adjust_for_weekend(date: datetime.date) -> datetime.date: # type: ignore
    if date.weekday() in [5, 6]:
        date = date + timedelta(days=(7 - date.weekday()))
    return date

def get_coupon_dates(issue_date, maturity_date) -> List[datetime.date]: # type: ignore
    payment_set = []
    payment_set.append(maturity_date)
    current_date = maturity_date

    while current_date > issue_date:
        current_date = maturity_date - relativedelta(months=len(payment_set) * 6)
        current_date = adjust_for_weekend(current_date)
        if current_date > issue_date:
            payment_set.append(current_date)
    payment_set.sort()
    return payment_set

def get_dates_and_cashflows(issue_date: datetime.date, maturity_date: datetime.date, as_of_date: datetime.date, coupon: float, get_days: bool = True, FV: int = 100):
    dates = get_coupon_dates(issue_date=issue_date, maturity_date=maturity_date)
    coupon_amt = coupon / 2
    return_list = list()
    if get_days:
        days = [(date - as_of_date).days for date in dates]
        for day in days[:-1]:
            return_list.append((day, coupon_amt))
        return_list.append((days[-1], coupon_amt + FV))
    else:
        for date in dates[:-1]:
            return_list.append((date, coupon_amt))
        return_list.append((dates[-1], coupon_amt + FV))
    return return_list

# Calculating bond price
def get_next_coupon_days(days_and_cashflows) -> int:
    for days, cashflow in days_and_cashflows:
        if days > 0:
            day = days
            return day

def get_last_coupon_days(days_and_cashflows, issue_date) -> int:
    current_day = -100000
    for day, cashflows in days_and_cashflows:
        if day < 0:
            if day > current_day:
                current_day = day
    if current_day == -100000:
        current_day = (issue_date - datetime.datetime.now().date()).days
    return current_day

def get_accrued(days_and_cashflows, coupon, issue_date) -> float:
    next_payment = get_next_coupon_days(days_and_cashflows=days_and_cashflows)
    last_payment = get_last_coupon_days(days_and_cashflows=days_and_cashflows, issue_date=issue_date)
    accrued = coupon / 2 * abs(last_payment) / (next_payment - last_payment)
    return accrued

def calculate_bond_price(issue_date: datetime.date,
                         maturity_date: datetime.date,
                         as_of_date: datetime.date,
                         coupon: float,
                         discount_rate: float,
                         dirty: bool = False) -> float:
    
    days_and_cashflows = get_dates_and_cashflows(issue_date=issue_date,
                                                 maturity_date=maturity_date,
                                                 as_of_date=as_of_date,
                                                 coupon=coupon,
                                                 get_days=True,
                                                 FV=100)
    # Dirty price
    PV = 0
    DISCOUNT_RATE = discount_rate / 100
    for day, cashflow in days_and_cashflows:
        if day > 0:
            pmt_value = cashflow/((1 + DISCOUNT_RATE)**(day/183))
            PV += pmt_value
    
    if not dirty:
        accrued = get_accrued(days_and_cashflows=days_and_cashflows, coupon=coupon, issue_date=issue_date)
        PV -= accrued
    return PV

def error_function(discount_rate: float, days_and_cashflows: List[tuple], price: float):
    calculated_price = calculate_bond_price(issue_date=issue_date,
                                            maturity_date=maturity_date,
                                            discount_rate=discount_rate,
                                            coupon=3.125,
                                            as_of_date=as_of_date,
                                            dirty=False)
    error = price - calculated_price
    return error

from scipy.optimize import newton
def get_ytm(price, coupon, issue_date, maturity_date):
    days_and_cashflows = get_dates_and_cashflows(issue_date, maturity_date, as_of_date, coupon, get_days=True)
    guess = coupon / 2 / 100

    try:
        ytm = newton(
            func=error_function,
            x0=guess,
            args=(days_and_cashflows, price)
        )
        return round(float(ytm * 2), 6)
    except RuntimeError:
        return None

get_ytm(88.9375, 3.125, issue_date, maturity_date)

4.032433

In [22]:
calculate_bond_price(issue_date=issue_date,
                     maturity_date=maturity_date,
                     as_of_date=as_of_date,
                     coupon=3.125,
                     discount_rate=1.5,
                     dirty=False)

101.74685124241614

In [24]:
import rateslib as rl

bond = rl.FixedRateBond(
    effective=datetime.datetime(2011, 11, 15),
    termination=datetime.datetime(2041, 11, 15),
    fixed_rate=3.125,
    spec="us_gb"  # US Government Bond
)
bond.ytm(price=88.9375, settlement=datetime.datetime(2024, 9, 23), dirty=False)

4.024213842228193